In [14]:
%pip install jupyter pandas scikit-learn matplotlib seaborn
%pip install nltk
%pip install umap-learn


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


#  Step 1: Load and Explore the Data



In [3]:
import pandas as pd

# Load product data (assuming it's a JSON file exported from MongoDB)
df = pd.read_json("../JSON/products.json")

# Preview the data
df.head()

,category,title,price,description
0,top,Silk Camisole,46.50,Featuring a modern cut and wool blend fabric t...
1,top,Crop Top,85.88,Soft and breathable modal construction with st...
2,top,Silk Camisole,107.06,Crafted from high-quality silk that's built to...
3,top,Patterned Blouse,136.74,Featuring a modern cut and polyester fabric th...
4,top,Tunic,33.05,Crafted from high-quality modal that's built t...


#  Step 2: Preprocess Data

In [6]:
df_encoded = pd.get_dummies(df, columns=["category"])
print(df_encoded.head())

              title   price  \
0     Silk Camisole   46.50   
1          Crop Top   85.88   
2     Silk Camisole  107.06   
3  Patterned Blouse  136.74   
4             Tunic   33.05   

                                         description  category_accessory  \
0  Featuring a modern cut and wool blend fabric t...               False   
1  Soft and breathable modal construction with st...               False   
2  Crafted from high-quality silk that's built to...               False   
3  Featuring a modern cut and polyester fabric th...               False   
4  Crafted from high-quality modal that's built t...               False   

   category_bottom  category_shoes  category_top  
0            False           False          True  
1            False           False          True  
2            False           False          True  
3            False           False          True  
4            False           False          True  


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_encoded["price"] = scaler.fit_transform(df_encoded[["price"]])

In [8]:
feature_columns = [col for col in df_encoded.columns if col not in ["title", "description", "_id"]]
X = df_encoded[feature_columns]


In [9]:
X.head()


,price,category_accessory,category_bottom,category_shoes,category_top
0,0.191659,False,False,False,True
1,0.399293,False,False,False,True
2,0.510967,False,False,False,True
3,0.667458,False,False,False,True
4,0.120742,False,False,False,True


#  Step 3: Fit the Nearest Neighbors Model

In [23]:
from sklearn.neighbors import NearestNeighbors

# Use cosine similarity for better results with mixed features
nn = NearestNeighbors(n_neighbors=5, metric='cosine')  # 5 because the first result will be the item itself
nn.fit(X)


,n_neighbors,5
,radius,1.0
,algorithm,'auto'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [25]:
# Helper function to get similar items
def get_similar_items(item_index, X, df, model, n=4):
    distances, indices = model.kneighbors([X.iloc[item_index]], n_neighbors=n+1)
    print("Cosine Distances:", distances[0][1:])
    # Skip the first one (it's the item itself)
    similar_indices = indices[0][1:]
    similar_items = df.iloc[similar_indices]
    
    return similar_items


In [19]:
# Test for random item
# Pick an item index (e.g. 0 for the first product)
item_index = 0
print("Target item:", df.iloc[item_index]["title"])

similar_items = get_similar_items(item_index, X, df, nn)
similar_items[["title", "category", "price"]]


Target item: Silk Camisole
Cosine Distances: [3.73874754e-07 5.81296696e-06 4.39722332e-04 4.62907321e-04
 5.75939601e-04]


C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,title,category,price
74,Tank Top,top,46.33
22,Patterned Blouse,top,45.83
55,Silk Camisole,top,40.70
42,Long Sleeve Thermal,top,52.52
12,Silk Camisole,top,53.22


# Add UMAP Coordinates

In [20]:
import umap.umap_ as umap

# Initialize and fit UMAP
reducer = umap.UMAP(n_neighbors=5, min_dist=0.3, metric='cosine')
embedding = reducer.fit_transform(X)

# Add UMAP coordinates to the DataFrame
df['umap_x'] = embedding[:, 0]
df['umap_y'] = embedding[:, 1]

# Preview
df[['title', 'category', 'price', 'umap_x', 'umap_y']].head()


,title,category,price,umap_x,umap_y
0,Silk Camisole,top,46.50,6.874131,14.280770
1,Crop Top,top,85.88,-0.037704,19.331036
2,Silk Camisole,top,107.06,-3.905039,16.346659
3,Patterned Blouse,top,136.74,14.120118,14.336964
4,Tunic,top,33.05,8.085502,14.706019


In [21]:
df_export = df[['title', 'category', 'price', 'umap_x', 'umap_y']]
df_export.to_json('../JSON/umap_products.json', orient='records', indent=2)


# Export Similar Items table for lookup

In [26]:
import json

similar_lookup = {}
for i in range(len(df)):
    similar = get_similar_items(i, X, df, nn)
    similar_lookup[df.iloc[i]["title"]] = similar["title"].tolist()

with open("../JSON/similar_items.json", "w") as f:
    json.dump(similar_lookup, f, indent=2)


C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have vali

Cosine Distances: [3.73874754e-07 5.81296696e-06 4.39722332e-04 4.62907321e-04]
Cosine Distances: [1.65682066e-06 6.43473097e-06 1.89370582e-05 1.95037160e-05]
Cosine Distances: [3.36868695e-06 4.80134350e-06 1.63799661e-04 2.79643628e-04]
Cosine Distances: [2.15539238e-06 6.88690554e-06 3.09408812e-04 3.47604569e-04]
Cosine Distances: [1.46241681e-05 3.93987171e-05 1.03205656e-04 2.01065861e-04]
Cosine Distances: [3.93987171e-05 1.02028888e-04 1.87048385e-04 2.70128822e-04]
Cosine Distances: [1.65682066e-06 9.39117170e-06 9.79145552e-06 1.46218249e-05]
Cosine Distances: [2.86077681e-05 8.65081609e-05 1.11670295e-04 1.94632170e-04]
Cosine Distances: [6.01935010e-07 1.34250367e-05 4.14875118e-05 4.51852838e-05]
Cosine Distances: [0.00012742 0.00019594 0.0005026  0.00051021]
Cosine Distances: [2.72365772e-05 1.11670295e-04 3.09408812e-04 3.63208227e-04]
Cosine Distances: [7.09216680e-05 7.37983041e-05 9.62130872e-05 1.95939996e-04]
Cosine Distances: [6.16916902e-06 5.94378276e-05 7.20024

C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have vali

Cosine Distances: [7.11400437e-05 8.03909854e-05 2.08499184e-04 3.03165617e-04]
Cosine Distances: [7.08952222e-08 1.12785686e-05 1.07088604e-04 1.31975652e-04]
Cosine Distances: [3.14337936e-06 7.26087538e-06 1.11796686e-05 2.51902960e-05]
Cosine Distances: [1.04259043e-06 2.65061621e-05 3.24529436e-05 4.81647056e-05]
Cosine Distances: [1.12785686e-05 1.31378613e-05 6.60934205e-05 1.87870919e-04]
Cosine Distances: [3.00675638e-07 1.54590289e-06 3.24529436e-05 4.51289970e-05]
Cosine Distances: [1.41461687e-06 1.90128621e-05 3.09468008e-05 6.12581002e-05]
Cosine Distances: [6.63745214e-09 2.47007324e-06 3.84329869e-06 7.61585476e-06]
Cosine Distances: [1.51155272e-07 1.41143533e-06 2.47007324e-06 2.73279639e-06]
Cosine Distances: [2.99600340e-05 9.82261763e-05 1.38638728e-04 2.08499184e-04]
Cosine Distances: [1.86209953e-07 1.67336072e-06 1.05403631e-04 1.16558181e-04]
Cosine Distances: [8.24615287e-06 5.17037486e-05 1.21102294e-04 1.33374972e-04]
Cosine Distances: [3.79783462e-05 6.7209

C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have vali

Cosine Distances: [2.24916182e-06 7.72013971e-06 1.40178871e-05 2.55092807e-05]
Cosine Distances: [9.39423765e-07 1.20173946e-05 3.04427487e-05 7.22971964e-04]
Cosine Distances: [1.58424662e-05 2.65566982e-05 2.88071406e-05 4.53797929e-05]
Cosine Distances: [3.96085892e-06 1.77391790e-05 2.04315284e-05 2.18673795e-05]
Cosine Distances: [8.09178978e-06 5.73675623e-05 9.28385535e-05 1.20229949e-04]
Cosine Distances: [0.00015816 0.00035092 0.00044602 0.00073188]
Cosine Distances: [2.50635488e-06 2.52441466e-05 2.65566982e-05 8.34214043e-05]
Cosine Distances: [0.00016397 0.00038095 0.00042153 0.00045744]
Cosine Distances: [6.70058329e-07 1.76399376e-05 3.44479330e-05 5.43012843e-05]
Cosine Distances: [8.84072745e-05 1.46716397e-04 1.74927085e-04 5.48593477e-04]
Cosine Distances: [3.87903554e-07 5.26649305e-07 4.87540187e-05 1.13076407e-04]
Cosine Distances: [5.56703200e-05 8.21209054e-05 8.34583316e-05 1.13133030e-04]
Cosine Distances: [1.19369401e-06 1.79790029e-05 5.32058800e-05 7.536306

C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have vali

Cosine Distances: [4.56865826e-05 6.25854261e-05 6.74446539e-05 7.33032031e-05]
Cosine Distances: [4.08072262e-06 1.57028495e-05 2.10478569e-04 2.22290571e-04]
Cosine Distances: [1.97410910e-06 6.27863816e-06 6.20583258e-05 1.03509234e-04]
Cosine Distances: [4.08072262e-06 3.57932974e-05 1.55947426e-04 2.08395941e-04]
Cosine Distances: [2.09935532e-05 2.19807767e-04 2.36412320e-04 3.22907659e-04]
Cosine Distances: [5.94574617e-07 7.98790149e-06 8.83890514e-06 1.11806594e-05]
Cosine Distances: [4.48675258e-07 3.06547169e-06 5.65661778e-06 3.35407385e-05]
Cosine Distances: [2.34903730e-05 5.01640496e-05 1.04727773e-03 1.22761259e-03]
Cosine Distances: [2.15333171e-08 1.74327008e-06 4.84855843e-06 8.83890514e-06]
Cosine Distances: [0.00019892 0.00036483 0.00037829 0.00043693]
Cosine Distances: [3.22991767e-09 3.02231885e-07 1.92134531e-05 2.27241846e-05]
Cosine Distances: [1.11806594e-05 1.69318531e-05 1.82139924e-05 3.80691753e-05]
Cosine Distances: [1.19644777e-06 1.48577922e-06 1.11204

C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\wasadmin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have vali